In [3]:
analysis_df = analysis_df.reset_index(drop=True)
analysis_df = analysis_df[analysis_df['check'] == 3]
analysis_df.columns = analysis_df.columns.str.lower()
len(analysis_df)
analysis_df

,experiment_duration,user_id,enthusiast,age,gender,education,region,openness,conscientiousness,extroversion,...,taste_extroversion,taste_neuroticism,taste_openness,taste_featurebased,listen_agreeableness,listen_conscientiousness,listen_extroversion,listen_neuroticism,listen_openness,listen_featurebased
0,0:15:15,68,3,3,1,6,4,3.50,3.75,3.50,...,4,4,4,4,4,4,4,4,4,4
1,0:10:11,69,5,4,1,4,4,4.25,3.00,4.25,...,4,3,5,5,5,3,4,3,5,5
2,0:22:20,70,4,4,1,6,5,3.25,3.50,2.25,...,3,3,2,3,4,3,4,4,1,2
3,0:07:40,71,5,2,1,2,5,4.75,3.00,3.50,...,4,4,4,4,3,4,4,4,4,4
4,0:10:08,73,4,4,1,4,2,4.00,4.00,3.75,...,5,4,5,5,4,5,5,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,0:13:02,790,4,3,1,3,5,4.25,4.25,3.50,...,4,5,4,3,4,3,4,5,4,3
376,0:27:14,791,4,7,1,6,4,4.25,4.00,3.50,...,3,5,2,4,2,4,3,5,2,4
377,0:11:16,792,5,1,1,3,2,5.00,3.00,2.00,...,5,3,4,5,5,4,5,3,4,5
378,0:14:58,799,2,2,1,2,5,4.00,3.50,1.75,...,4,3,4,3,2,4,4,3,4,3


In [5]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.miscmodels.ordinal_model import OrderedModel
import re

def normalize_expl_type(x: str) -> str:
    if pd.isna(x):
        return ""
    s = re.sub(r"[^a-z]", "", str(x).strip().lower())
    mapping = {
        "feature": "featurebased", "featurebased": "featurebased", "features": "featurebased",
        "openness": "openness", "open": "openness", "o": "openness",
        "conscientiousness": "conscientiousness", "cons": "conscientiousness", "c": "conscientiousness",
        "extraversion": "extroversion", "extroversion": "extroversion", "extra": "extroversion", "e": "extroversion",
        "agreeableness": "agreeableness", "agree": "agreeableness", "a": "agreeableness",
        "neuroticism": "neuroticism", "neuro": "neuroticism", "n": "neuroticism",
    }
    return mapping.get(s, s)

def to_ordered_numeric(s: pd.Series, mapping: dict | None = None) -> pd.Series:
    """Coerce to ordered numeric. If already numeric, keep it; else code by categorical order or provided mapping."""
    if mapping is not None:
        return s.astype(str).str.strip().str.lower().map(mapping)
    sn = pd.to_numeric(s, errors="coerce")
    if sn.notna().any():
        return sn
    cats = pd.Categorical(s.astype(str))
    return pd.Series(cats.codes, index=s.index).replace(-1, np.nan)

def _format_ols_table(res, decimals=3):
    ci = res.conf_int()
    tbl = pd.DataFrame({
        "coef": res.params,
        "t": res.tvalues,
        "p>|t|": res.pvalues,
        "ci_low": ci[0],
        "ci_high": ci[1],
    })
    return tbl.round(decimals)

def fit_ols(formula: str, data: pd.DataFrame, cluster_col="user_id"):
    model = smf.ols(formula=formula, data=data)
    return model.fit(cov_type="cluster", cov_kwds={"groups": data[cluster_col]})


df = analysis_df.copy()
df["explanation_type_norm"] = df["explanation type"].apply(normalize_expl_type)


styles = ["openness","conscientiousness","extroversion","agreeableness","neuroticism","featurebased"]
traits_ord = ["openness","conscientiousness","extroversion","agreeableness","neuroticism"]


for target_prefix in ["quality", "taste", "listen"]:
    ycol = f"y_{target_prefix}"
    df[ycol] = np.nan
    for st in styles:
        src_col = f"{target_prefix}_{st}"
        if src_col in df.columns:
            mask = df["explanation_type_norm"] == st
            df.loc[mask, ycol] = df.loc[mask, src_col]

model_df = df.dropna(subset=["y_quality","y_taste","y_listen"]).copy()


for t in traits_ord:
    if t in model_df.columns:
        model_df[t] = pd.to_numeric(model_df[t], errors="coerce")


model_df["edu_num"] = to_ordered_numeric(model_df.get("education", pd.Series(np.nan, index=model_df.index)))


CAT_DEMOS = [c for c in ["gender","region"] if c in model_df.columns]
NUM_DEMOS = []
if "enthusiast" in model_df.columns and model_df["enthusiast"].dropna().dtype.kind in "biufc":
    NUM_DEMOS.append("enthusiast")



for t in traits_ord:
    model_df[f"{t}_c"] = pd.to_numeric(model_df[t], errors="coerce") - 3


for v in ["edu_num"] + NUM_DEMOS:
    if v in model_df.columns:
        mv = pd.to_numeric(model_df[v], errors="coerce")
        model_df[f"{v}_c"] = mv - mv.mean()

traits_c = [f"{t}_c" for t in traits_ord]
num_covs_c = [v for v in ["edu_num_c"] if v in model_df.columns] + [f"{v}_c" for v in NUM_DEMOS if f"{v}_c" in model_df.columns]


style_term = "C(explanation_type_norm, Treatment('featurebased'))"

# Model 1: style only
m1_terms = [style_term]

# Model 2: style + centered traits + centered covariates + categorical demos
m2_terms = [style_term] + traits_c + num_covs_c + [f"C({c})" for c in CAT_DEMOS]

# Model 3: Model 2 + style×trait interactions (traits are centered)
m3_interactions = [f"{style_term}:{t}" for t in traits_c]
m3_terms = m2_terms + m3_interactions

def run_block(dv: str, terms: list, label: str):
    formula = f"{dv} ~ " + " + ".join(terms)
    res = fit_ols(formula, model_df)
    print("\n" + "="*88)
    print(f"{label}  |  DV = {dv}")
    print("="*88)
    print(_format_ols_table(res, decimals=3).to_string())
    print("\nR^2: {:.3f}   Adj. R^2: {:.3f}   N: {}".format(res.rsquared, res.rsquared_adj, int(res.nobs)))
    return res


ols_results = {}
for dv in ["y_quality","y_taste","y_listen"]:
    r1 = run_block(dv, m1_terms, label="BASELINE (style only)")
    r2 = run_block(dv, m2_terms, label="Model 2 (style + centered traits + covariates)")
    r3 = run_block(dv, m3_terms, label="Model 3 (style×traits + covariates)")
    ols_results[dv] = (r1, r2, r3)


def print_sig_only(res, title, alpha=0.05):
    tbl = _format_ols_table(res)
    sig = tbl[tbl["p>|t|"] < alpha]
    if sig.empty:
        print(f"\n{title}: no terms with p<{alpha}")
    else:
        print(f"\n{title}: significant terms (p<{alpha})")
        print(sig.sort_values("p>|t|")[["coef","p>|t|"]].to_string())

for dv in ["y_quality","y_taste","y_listen"]:
    print_sig_only(ols_results[dv][2], f"Model 3 significant terms for {dv}")


def fit_ordered_logit(dv: str):
    rhs_cols = ["explanation_type_norm"] + CAT_DEMOS + traits_c + num_covs_c
    rhs_cols = [c for c in rhs_cols if c in model_df.columns]
    subset = model_df[[dv] + rhs_cols].dropna()

    X = pd.get_dummies(subset[["explanation_type_norm"] + CAT_DEMOS], drop_first=True)
    num_cols = [c for c in traits_c + num_covs_c if c in subset.columns]
    if num_cols:
        X = pd.concat([X, subset[num_cols].apply(pd.to_numeric, errors="coerce")], axis=1)

    X = X.apply(pd.to_numeric, errors="coerce").astype(float)
    y = pd.to_numeric(subset[dv], errors="coerce").round().clip(1,5).astype(int)

    mod = OrderedModel(y, X, distr="logit")
    res = mod.fit(method="bfgs", disp=False)

    # Null model for McFadden pseudo-R^2
    X0 = np.ones((len(y), 1))
    mod0 = OrderedModel(y, X0, distr="logit")
    res0 = mod0.fit(method="bfgs", disp=False)

    llf_full, llf_null = float(res.llf), float(res0.llf)
    k = X.shape[1]
    mcfadden = 1.0 - (llf_full / llf_null)
    mcfadden_adj = 1.0 - ((llf_full - k) / llf_null)

    print("\n" + "-"*88)
    print(f"Ordered logit robustness | DV = {dv}")
    print("-"*88)
    kfeat = X.shape[1]
    params = pd.Series(res.params[:kfeat], index=X.columns, name="coef")
    zvals  = params / pd.Series(res.bse[:kfeat], index=X.columns)
    pvals  = pd.Series(res.pvalues[:kfeat], index=X.columns, name="p>|z|")
    coef_tbl = pd.concat([params, zvals.rename("z"), pvals], axis=1)
    coef_tbl["odds_ratio"] = np.exp(coef_tbl["coef"])
    print(coef_tbl[["coef","z","p>|z|","odds_ratio"]].round(3).to_string())
    print("\nPseudo-R^2 (McFadden): {:.3f}   Adjusted McFadden: {:.3f}   N: {}".format(mcfadden, mcfadden_adj, len(y)))
    return res, (mcfadden, mcfadden_adj)

def style_contrast_at_means(res, style_label="neuroticism"):

    params = res.params
    index = list(params.index)


    means_unc = model_df[traits_ord].mean()
    means_c = means_unc - 3

    def idx(name):
        return index.index(name) if name in index else None

    L = np.zeros(len(params))
    main_name = f"C(explanation_type_norm, Treatment('featurebased'))[T.{style_label}]"
    i = idx(main_name)
    if i is None:
        raise ValueError(f"Style main effect not found: {main_name}")
    L[i] = 1.0


    for t in traits_ord:
        term = f"C(explanation_type_norm, Treatment('featurebased'))[T.{style_label}]:{t}_c"
        j = idx(term)
        if j is not None:
            L[j] = means_c[t]

    test = res.t_test(L)
    return test


BASELINE (style only)  |  DV = y_quality
                                                                           coef       t  p>|t|  ci_low  ci_high
Intercept                                                                 4.674  79.890  0.000   4.560    4.789
C(explanation_type_norm, Treatment('featurebased'))[T.agreeableness]     -0.036  -0.406  0.685  -0.213    0.140
C(explanation_type_norm, Treatment('featurebased'))[T.conscientiousness] -0.320  -2.294  0.022  -0.593   -0.047
C(explanation_type_norm, Treatment('featurebased'))[T.extroversion]      -0.297  -2.535  0.011  -0.527   -0.067
C(explanation_type_norm, Treatment('featurebased'))[T.neuroticism]        0.020   0.236  0.814  -0.146    0.186
C(explanation_type_norm, Treatment('featurebased'))[T.openness]          -0.049  -0.490  0.624  -0.247    0.148

R^2: 0.046   Adj. R^2: 0.032   N: 348

Model 2 (style + centered traits + covariates)  |  DV = y_quality
                                                                    